In [ ]:
#using tutorial https://www.tensorflow.org/tutorials/text/text_generation#advanced_customized_training 

In [ ]:
import pandas as pd
import numpy as np

#more imports
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

import random
import sys
import os
import time

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#for saving the model
!pip install -q pyyaml h5py

In [ ]:
# Read in the data
df = pd.read_csv("ufo_update.csv")
text_ = " "

In [ ]:
df.head()

,text,stats,date_time,report_link,city,state,shape,duration,summary,posted
0,The background is a blue sky. The shape of th...,Occurred : 8/5/2020 10:15 (Entered as : 08/05...,8/5/20 10:15,http://www.nuforc.org/webreports/158/S158217.html,Ty Ty,GA,Unknown,5 minutes,The background is a blue sky. The shape of the...,8/6/20
1,Black polygonal craft with lights wobbling thr...,Occurred : 8/5/2020 18:50 (Entered as : 08/05...,8/5/20 18:50,http://www.nuforc.org/webreports/158/S158226.html,New Braunfels,TX,Chevron,2-3 minutes,Black polygonal craft with lights wobbling thr...,8/6/20
2,I observed a white ball of light flying from t...,Occurred : 8/4/2020 20:30 (Entered as : 08/04...,8/4/20 20:30,http://www.nuforc.org/webreports/158/S158228.html,Mesa,AZ,Circle,2 minutes,I observed a white ball of light flying from t...,8/6/20
3,High altitude fast moving delta shaped flames ...,Occurred : 8/4/2020 22:00 (Entered as : 08/04...,8/4/20 22:00,http://www.nuforc.org/webreports/158/S158221.html,Miami Beach,FL,Fireball,1 minute,High altitude fast moving delta shaped flames....,8/6/20
4,Three unmistakable lights moving at a speed th...,Occurred : 8/5/2020 04:33 (Entered as : 08/05...,8/5/20 04:33,http://www.nuforc.org/webreports/158/S158220.html,Pheonix,AZ,Triangle,3 seconds,Three unmistakable lights moving at a speed th...,8/6/20


In [ ]:
df['text'][0]

'The background is a blue sky.  The shape of the UFO is indescribable. UFO was moving from north to south at about tree top level at a very slow speed as if looking for something.'

In [ ]:
df.shape

(93817, 10)

In [ ]:
#sub = df.sample(frac=.5)

sub = df.iloc[:1000]

In [ ]:
sub.head()

,text,stats,date_time,report_link,city,state,shape,duration,summary,posted
0,The background is a blue sky. The shape of th...,Occurred : 8/5/2020 10:15 (Entered as : 08/05...,8/5/20 10:15,http://www.nuforc.org/webreports/158/S158217.html,Ty Ty,GA,Unknown,5 minutes,The background is a blue sky. The shape of the...,8/6/20
1,Black polygonal craft with lights wobbling thr...,Occurred : 8/5/2020 18:50 (Entered as : 08/05...,8/5/20 18:50,http://www.nuforc.org/webreports/158/S158226.html,New Braunfels,TX,Chevron,2-3 minutes,Black polygonal craft with lights wobbling thr...,8/6/20
2,I observed a white ball of light flying from t...,Occurred : 8/4/2020 20:30 (Entered as : 08/04...,8/4/20 20:30,http://www.nuforc.org/webreports/158/S158228.html,Mesa,AZ,Circle,2 minutes,I observed a white ball of light flying from t...,8/6/20
3,High altitude fast moving delta shaped flames ...,Occurred : 8/4/2020 22:00 (Entered as : 08/04...,8/4/20 22:00,http://www.nuforc.org/webreports/158/S158221.html,Miami Beach,FL,Fireball,1 minute,High altitude fast moving delta shaped flames....,8/6/20
4,Three unmistakable lights moving at a speed th...,Occurred : 8/5/2020 04:33 (Entered as : 08/05...,8/5/20 04:33,http://www.nuforc.org/webreports/158/S158220.html,Pheonix,AZ,Triangle,3 seconds,Three unmistakable lights moving at a speed th...,8/6/20


In [ ]:
sub['text'] = sub['text'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
text = ' '.join(sub['text'].tolist())

In [ ]:
#Clean text

import re
from bs4 import BeautifulSoup

def cleanText(txt):
    txt = BeautifulSoup(txt).get_text()
    txt = txt.replace('\t', '')
    txt = txt.replace('\x9d', '')
    txt = txt.replace('\xa0', '')
    #remove all non letters from text
    txt = re.sub("[^A-Za-z0-9_.,!'/$]", " ", txt) 
    txt = txt.split()
    txt = " ".join(txt)

    return txt

In [ ]:
text = cleanText(text)

In [ ]:
# Encode the data as Chars

#get all unique chars
chars = sorted(list(set(text)))
print('unique characters', format(len(chars)))

unique characters 70


In [ ]:
#create a mapping from unique characters to indices

char_indices = {u:i for i, u in enumerate(chars)}
indices_char = np.array(chars)

text_as_int = np.array([char_indices[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char_indices, range(70)):
    print('  {:4s}: {:3d},'.format(repr(char), char_indices[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '$' :   2,
  "'" :   3,
  ',' :   4,
  '.' :   5,
  '/' :   6,
  '0' :   7,
  '1' :   8,
  '2' :   9,
  '3' :  10,
  '4' :  11,
  '5' :  12,
  '6' :  13,
  '7' :  14,
  '8' :  15,
  '9' :  16,
  'A' :  17,
  'B' :  18,
  'C' :  19,
  'D' :  20,
  'E' :  21,
  'F' :  22,
  'G' :  23,
  'H' :  24,
  'I' :  25,
  'J' :  26,
  'K' :  27,
  'L' :  28,
  'M' :  29,
  'N' :  30,
  'O' :  31,
  'P' :  32,
  'Q' :  33,
  'R' :  34,
  'S' :  35,
  'T' :  36,
  'U' :  37,
  'V' :  38,
  'W' :  39,
  'X' :  40,
  'Y' :  41,
  'Z' :  42,
  '_' :  43,
  'a' :  44,
  'b' :  45,
  'c' :  46,
  'd' :  47,
  'e' :  48,
  'f' :  49,
  'g' :  50,
  'h' :  51,
  'i' :  52,
  'j' :  53,
  'k' :  54,
  'l' :  55,
  'm' :  56,
  'n' :  57,
  'o' :  58,
  'p' :  59,
  'q' :  60,
  'r' :  61,
  's' :  62,
  't' :  63,
  'u' :  64,
  'v' :  65,
  'w' :  66,
  'x' :  67,
  'y' :  68,
  'z' :  69,
  ...
}


In [ ]:
# The maximum length sentence you want for a single input in characters

seq_length = 100   #try adjusting sequence length
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(indices_char[i.numpy()])   #do not alias numpy as np

T
h
e
 
b


In [ ]:
#The batch method lets us easily convert these individual characters to sequences of the desired size.

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(indices_char[item.numpy()])))  #do not alias numpy as np

'The background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to sou'
'th at about tree top level at a very slow speed as if looking for something. Black polygonal craft wi'
'th lights wobbling through sky. I was in the back yard area of my apartment complex with only my dog '
'when I saw what I thought was a drone. 1 solid black object that appeared to be a pentagonal pyramid '
'flying slowly through the air. It looked to be about the size of a small car and less than 1000ft or '


In [ ]:
#For each sequence, duplicate and shift it to form the input and target text by 
#using the map method to apply a simple function to each batch:

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
#Print the first example input and target values:

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(indices_char[input_example.numpy()])))
    print('Target data:', repr(''.join(indices_char[target_example.numpy()])))

Input data:  'The background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to so'
Target data: 'he background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to sou'


In [ ]:
#each index of these vectors is processed as a one time step. For the input at
#time step 0, the model receives the index for 'T' and tries to predict the
#index for i as the next character. At the next timestep, it does the same thing
#but the RNN considers the previous step context in addition to the current input
#character

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(indices_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(indices_char[target_idx])))

Step    0
  input: 36 ('T')
  expected output: 51 ('h')
Step    1
  input: 51 ('h')
  expected output: 48 ('e')
Step    2
  input: 48 ('e')
  expected output: 0 (' ')
Step    3
  input: 0 (' ')
  expected output: 45 ('b')
Step    4
  input: 45 ('b')
  expected output: 44 ('a')


In [ ]:
#Create training batches - you can use tf.data to split the text into manageable
#sequences. But before feeding this data into the model, you need to shuffle the
#data and pack it into batches

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build the model
For each character the model looks up the embedding, runs the GRU one timestep
with the embedding as input, and applies the dense layer to generate logits
predicting the log-likelihood of the next character

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(chars)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(chars),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
#check the shape of the output

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length,vocab_size)")

(64, 100, 70) # (batch_size, sequence_length,vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           17920     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 70)            71750     
Total params: 4,027,974
Trainable params: 4,027,974
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#to get prediction from the model you need to sample from the output 
#distribution to get actual character indices

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([40, 41, 30, 64, 20,  7,  6, 58, 11, 22, 26, 10,  4, 49, 69, 43, 22,
       64, 27, 62, 67, 34, 55,  4, 16, 17, 29, 17,  0, 53, 54, 60, 62, 11,
       26,  1, 45, 41, 64, 29, 69, 25, 46,  1, 13, 27, 23, 38, 41, 15, 46,
       28,  3, 21, 13, 45, 14, 20, 14, 28, 62, 64, 63, 39, 24, 10, 18, 55,
        0, 48, 12,  0, 16,  9, 61, 46,  7, 15, 69, 14, 20, 38,  8, 53, 66,
       41, 60,  6, 15,  2, 53, 19, 30, 27,  0, 36, 13,  7, 48, 62])

In [ ]:
#decode these to see the text predicted by this untrained model

print("Input: \n", repr("".join(indices_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(indices_char[sampled_indices ])))

Input: 
 'ow frequently as a child. Over the years I have been bedazzled by dozens of sights similar to this o'

Next Char Predictions: 
 "XYNuD0/o4FJ3,fz_FuKsxRl,9AMA jkqs4J!bYuMzIc!6KGVY8cL'E6b7D7LsutWH3Bl e5 92rc08z7DV1jwYq/8$jCNK T60es"


In [ ]:
#train the model - at this point problem can be treated as a standard
#classification problem. Given the previous RNN state, and the input this time
#step, predict the class of the next character

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 70)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.249167


In [ ]:
#Configure the training procedure using the tf.keras.Model.compile method. 
#Use tf.keras.optimizers.Adam with default arguments and the loss function.

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
#Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints 
#are saved during training:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
97/97 [==============================] - 6s 42ms/step - loss: 3.5509 - accuracy: 0.1937
Epoch 2/30
97/97 [==============================] - 5s 41ms/step - loss: 2.1786 - accuracy: 0.3785
Epoch 3/30
97/97 [==============================] - 5s 41ms/step - loss: 1.8654 - accuracy: 0.4626
Epoch 4/30
97/97 [==============================] - 5s 41ms/step - loss: 1.6108 - accuracy: 0.5406
Epoch 5/30
97/97 [==============================] - 5s 41ms/step - loss: 1.4606 - accuracy: 0.5802
Epoch 6/30
97/97 [==============================] - 5s 41ms/step - loss: 1.3536 - accuracy: 0.6087
Epoch 7/30
97/97 [==============================] - 5s 41ms/step - loss: 1.2830 - accuracy: 0.6272
Epoch 8/30
97/97 [==============================] - 5s 41ms/step - loss: 1.2256 - accuracy: 0.6408
Epoch 9/30
97/97 [==============================] - 5s 41ms/step - loss: 1.1801 - accuracy: 0.6532
Epoch 10/30
97/97 [==============================] - 5s 41ms/step - loss: 1.1305 - accuracy: 0.6663
Epoch 11/

Generate text - to keep prediction step simple use a batch size of 1
because of the way the RNN state is passed from timestep to timestep, the 
model only accepts a fixed batch size once built. To run the model with a diff
batch size, you need to rebuild the model and restore the weights from the checkpoint. 

Looking at the genearated text, the model knows when to capitalize, make
paragraphs and imitate the vocabulary. With a small # of epochs it has not learned to form coherent sentences

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            17920     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 70)             71750     
Total params: 4,027,974
Trainable params: 4,027,974
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#save those weights

model.save_weights(checkpoint_dir.format(epoch=0))

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'./training_checkpoints/ckpt_30'

In [ ]:
#create and train new model instance

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)


In [ ]:
model.fit(dataset, epochs=5)

In [ ]:
#save entire model to HDF5 file
#the '.h5' extension indicates that the model should be saved to HDF5
model.save('my_model.hf')

The prediction loop: the following code generates the text.
begin by choosing a start string, initializing the RNN state and setting the 
number of characters to generate. Get the prediction distribution of the next
character using the start string and the RNN state. Then use a categorical 
distribution to calculate the index of the predicted character. Use this
predicted character as our next input to the model. The RNN state returned by 
the model is fed back into the model so that it now has more context, instead
only one character. After predicting the next character, the modified RNN states
are again fed back into the model, which is how it learns more context from the
previously predicted characters

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char_indices[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(indices_char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"I saw a little green man "))

SAVE the model

In [ ]:
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [ ]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
# Train the model with the new callback
model.fit(dataset,
          epochs=EPOCHS,
          callbacks=[checkpoint_callback])  # Pass callback to training

In [ ]:
# model = build_model(
#     vocab_size=len(chars),
#     embedding_dim=embedding_dim,
#     rnn_units=rnn_units,
#     batch_size=BATCH_SIZE)

In [ ]:
# optimizer = tf.keras.optimizers.Adam()

In [ ]:
# @tf.function
# def train_step(inp, target):
#     with tf.GradientTape() as tape:
#         predictions = model(inp)
#         loss = tf.reduce_mean(
#             tf.keras.losses.sparse_categorical_crossentropy(
#                 target, predictions, from_logits=True))
#     grads = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(grads, model.trainable_variables))

#     return loss

In [ ]:
# Training step
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     start = time.time()

#     # resetting the hidden state at the start of every epoch
#     model.reset_states()

#     for (batch_n, (inp, target)) in enumerate(dataset):
#         loss = train_step(inp, target)

#         if batch_n % 100 == 0:
#             template = 'Epoch {} Batch {} Loss {}'
#             print(template.format(epoch + 1, batch_n, loss))

#     # saving (checkpoint) the model every 5 epochs
#     if (epoch + 1) % 5 == 0:
#         model.save_weights(checkpoint_prefix.format(epoch=epoch))

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# model.save_weights(checkpoint_prefix.format(epoch=epoch))

In [ ]:
# from keras.optimizers import RMSprop
# from keras.layers import Dense, Activation  #move later

# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars)))
# model.add(Activation('softmax'))
# optimizer = RMSprop(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temperature
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)

In [ ]:
# def on_epoch_end(epoch, _):
#     # Function invoked at end of each epoch. Prints generated text.
#     print()
#     print('----- Generating text after Epoch: %d' % epoch)

#     start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(400):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
# history_2 = model.fit(x, y,
#           batch_size=128,
#           epochs=100,
#           callbacks=[print_callback])

In [ ]:
# history_3 = model.fit(x, y,
#           batch_size=128,
#           epochs=150,
#           callbacks=[print_callback])